In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

#Image Preprocessing

##Import from roboflow

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="EIYLr8taYjmOnw7xykZu")
project = rf.workspace("neural-flow").project("hit-and-run-case")
version = project.version(12)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to hit-and-run-case-12 in yolov5pytorch:: 100%|██████████| 7816/7816 [00:02<00:00, 3168.15it/s]


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time

# Input and output directories
IMAGE_DIR = "/content/hit-and-run-case-12/valid/images/"
LABEL_DIR = "/content/hit-and-run-case-12/valid/labels/"
OUTPUT_IMAGE_DIR = "/content/drive/MyDrive/pre-processed/valid/augmented_images/"
OUTPUT_LABEL_DIR = "/content/drive/MyDrive/pre-processed/valid/augmented_labels/"

os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
os.makedirs(OUTPUT_LABEL_DIR, exist_ok=True)

def convert_yolo_to_bbox(img_width, img_height, x_center, y_center, width, height):
    """Convert YOLO format to pixel coordinates"""
    x1 = max(0, int((x_center - width / 2) * img_width))
    y1 = max(0, int((y_center - height / 2) * img_height))
    x2 = min(img_width, int((x_center + width / 2) * img_width))
    y2 = min(img_height, int((y_center + height / 2) * img_height))
    return x1, y1, x2, y2

def convert_bbox_to_yolo(img_width, img_height, x1, y1, x2, y2):
    """Convert pixel coordinates to YOLO format"""
    x_center = ((x1 + x2) / 2) / img_width
    y_center = ((y1 + y2) / 2) / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return np.clip(x_center, 0, 1), np.clip(y_center, 0, 1), np.clip(width, 0, 1), np.clip(height, 0, 1)

def read_yolo_annotation(label_path):
    """Read YOLO format annotations from file"""
    annotations = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:])
                    annotations.append((class_id, x_center, y_center, width, height))
    return annotations

def write_yolo_annotation(label_path, annotations):
    """Write YOLO format annotations to file"""
    with open(label_path, "w") as f:
        for ann in annotations:
            f.write(f"{ann[0]} {ann[1]:.6f} {ann[2]:.6f} {ann[3]:.6f} {ann[4]:.6f}\n")

def draw_bboxes(img, bboxes):
    """Draw bounding boxes on image"""
    img_copy = img.copy()
    for class_id, x_center, y_center, width, height in bboxes:
        x1, y1, x2, y2 = convert_yolo_to_bbox(img.shape[1], img.shape[0], x_center, y_center, width, height)
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_copy, f"Class {class_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return img_copy

def apply_augmentation(image, annotations, transformation, target_class, saturation_value=None, hue_value=None):
    """Apply selected augmentation with single value parameters"""
    h, w = image.shape[:2]
    new_annotations = []

    if transformation == "flip_horizontal":
        image = cv2.flip(image, 1)
        for class_id, x_center, y_center, width, height in annotations:
            new_x = 1 - x_center
            new_annotations.append((class_id, new_x, y_center, width, height))

    elif transformation == "flip_vertical":
        image = cv2.flip(image, 0)
        for class_id, x_center, y_center, width, height in annotations:
            new_y = 1 - y_center
            new_annotations.append((class_id, x_center, new_y, width, height))

    elif transformation == "rotate_90":
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        for class_id, x_center, y_center, width, height in annotations:
            x1, y1, x2, y2 = convert_yolo_to_bbox(w, h, x_center, y_center, width, height)
            new_x1 = y1
            new_y1 = w - x2
            new_x2 = y2
            new_y2 = w - x1
            x_center, y_center, width, height = convert_bbox_to_yolo(h, w, new_x1, new_y1, new_x2, new_y2)
            new_annotations.append((class_id, x_center, y_center, width, height))

    elif transformation == "scale":
        scale_factor = 1.2
        new_w, new_h = int(w * scale_factor), int(h * scale_factor)
        image = cv2.resize(image, (new_w, new_h))
        start_x = (new_w - w) // 2
        start_y = (new_h - h) // 2
        image = image[start_y:start_y + h, start_x:start_x + w]

        for class_id, x_center, y_center, width, height in annotations:
            new_x = (x_center * scale_factor - start_x/w) * (w/w)
            new_y = (y_center * scale_factor - start_y/h) * (h/h)
            new_w = width * scale_factor * (w/w)
            new_h = height * scale_factor * (h/h)
            new_annotations.append((class_id, np.clip(new_x, 0, 1), np.clip(new_y, 0, 1),
                                 np.clip(new_w, 0, 1), np.clip(new_h, 0, 1)))

    elif transformation == "saturation":
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv[..., 1] = hsv[..., 1] * saturation_value  # Adjust saturation
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        new_annotations = annotations.copy()

    elif transformation == "hue":
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv[..., 0] = (hsv[..., 0] + hue_value) % 180  # Adjust hue
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        new_annotations = annotations.copy()

    return image, new_annotations

def get_valid_value_input(prompt, min_val, max_val):
    """Get and validate a single value input from user"""
    while True:
        try:
            value_str = input(prompt).strip()
            if not value_str:
                return None

            value = float(value_str)
            if min_val <= value <= max_val:
                return value
            else:
                print(f"Value must be between {min_val} and {max_val}")
        except ValueError:
            print(f"Please enter a number between {min_val} and {max_val} or press Enter to skip")

def get_user_augmentation_choices():
    """Get user choices for augmentations with single value parameters"""
    print("\nAvailable augmentations:")
    print("1. Horizontal Flip")
    print("2. Vertical Flip")
    print("3. 90° Rotation")
    print("4. Scale")
    print("5. Saturation Adjustment")
    print("6. Hue Adjustment")

    choices = input("\nEnter the numbers of augmentations you want to apply (separated by spaces): ").strip().split()
    selected_augmentations = []

    augmentation_map = {
        "1": "flip_horizontal",
        "2": "flip_vertical",
        "3": "rotate_90",
        "4": "scale",
        "5": "saturation",
        "6": "hue"
    }

    for choice in choices:
        if choice in augmentation_map:
            selected_augmentations.append(augmentation_map[choice])

    # Get single values for saturation and hue if selected
    saturation_value = None
    hue_value = None

    if "saturation" in selected_augmentations:
        print("\nSaturation Adjustment:")
        print("Enter the saturation value (0-180)")
        print("- 0 means no saturation (grayscale)")
        print("- 180 means maximum saturation")
        saturation_value = get_valid_value_input("Enter saturation value: ", 0, 180)

    if "hue" in selected_augmentations:
        print("\nHue Adjustment:")
        print("Enter the hue value (0-180)")
        print("- 0/180 is red")
        print("- 60 is green")
        print("- 120 is blue")
        hue_value = get_valid_value_input("Enter hue value: ", 0, 180)

    return selected_augmentations, saturation_value, hue_value

def main():
    """Main function to run the augmentation process"""
    # Get the target class ID
    target_class = int(input("Enter the target class ID for augmentation: ").strip())

    # Get user choices for augmentations
    selected_augmentations, saturation_value, hue_value = get_user_augmentation_choices()

    if not selected_augmentations:
        print("No valid augmentations selected. Process aborted.")
        return

    # Ask if user wants to view results
    view_results = input("Do you want to view augmentation results? (y/n): ").strip().lower() == 'y'

    save_all = input("Do you want to save all augmented images? (y/n): ").strip().lower()

    # Process all images
    for filename in os.listdir(IMAGE_DIR):
        if not (filename.endswith(".jpg") or filename.endswith(".png")):
            continue

        image_path = os.path.join(IMAGE_DIR, filename)
        label_path = os.path.join(LABEL_DIR, filename.rsplit(".", 1)[0] + ".txt")

        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {filename}")
            continue

        annotations = read_yolo_annotation(label_path)
        if not any(ann[0] == target_class for ann in annotations):
            continue

        for transformation in selected_augmentations:
            aug_image, aug_annotations = apply_augmentation(
                image.copy(),
                annotations,
                transformation,
                target_class,
                saturation_value if transformation == "saturation" else None,
                hue_value if transformation == "hue" else None
            )

            # Display augmented image with bounding boxes
            display_img = draw_bboxes(aug_image, aug_annotations)

            if view_results:
                # Draw original image with annotations
                original_display = draw_bboxes(image, annotations)

                # Create figure for side-by-side display
                plt.figure(figsize=(20, 10))

                # Original image subplot
                plt.subplot(1, 2, 1)
                plt.imshow(cv2.cvtColor(original_display, cv2.COLOR_BGR2RGB))
                plt.title(f"Original: {filename}")
                plt.axis('off')

                # Augmented image subplot
                plt.subplot(1, 2, 2)
                plt.imshow(cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB))
                title = f"{filename} - {transformation}"
                if transformation == "saturation" and saturation_value is not None:
                    title += f" (saturation: {saturation_value:.1f})"
                elif transformation == "hue" and hue_value is not None:
                    title += f" (hue: {hue_value:.1f})"
                plt.title(title)
                plt.axis('off')

                plt.show()
                plt.close()

            if save_all == "y":
                # Include parameters in filename
                param_str = ""
                if transformation == "saturation" and saturation_value is not None:
                    param_str = f"_sat{saturation_value:.1f}"
                elif transformation == "hue" and hue_value is not None:
                    param_str = f"_hue{hue_value:.1f}"

                aug_image_name = f"{filename.rsplit('.', 1)[0]}_{transformation}{param_str}.jpg"
                aug_label_name = f"{filename.rsplit('.', 1)[0]}_{transformation}{param_str}.txt"

                cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, aug_image_name), aug_image)
                write_yolo_annotation(os.path.join(OUTPUT_LABEL_DIR, aug_label_name), aug_annotations)

    print("Augmentation process completed successfully!")

if __name__ == "__main__":
    main()

Enter the target class ID for augmentation: 2

Available augmentations:
1. Horizontal Flip
2. Vertical Flip
3. 90° Rotation
4. Scale
5. Saturation Adjustment
6. Hue Adjustment

Enter the numbers of augmentations you want to apply (separated by spaces): 5

Saturation Adjustment:
Enter the saturation value (0-180)
- 0 means no saturation (grayscale)
- 180 means maximum saturation
Enter saturation value: 3
Do you want to view augmentation results? (y/n): n
Do you want to save all augmented images? (y/n): y
Augmentation process completed successfully!


In [ ]:
from roboflow import Roboflow
import os

# Roboflow setup
rf = Roboflow(api_key="EIYLr8taYjmOnw7xykZu")
project = rf.workspace("neural-flow").project("hit-and-run-case")

# Define paths
image_dir = "/content/drive/MyDrive/pre-processed/valid/augmented_images/"
label_dir = "/content/drive/MyDrive/pre-processed/valid/augmented_labels/"

# Process and upload images
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, filename.rsplit(".", 1)[0] + ".txt")

        if os.path.exists(label_path):  # Ensure annotation file exists
            project.upload(image_path, annotation_path=label_path, num_retry_uploads=3)
            print(f"Uploaded {filename} with annotation.")
        else:
            print(f"Skipping {filename}, annotation file missing.")


loading Roboflow workspace...
loading Roboflow project...
Uploaded Bangladesh_Army_Nissan_Sunny_N16_staff-car_-29448969845-_jpg.rf.83cb452e13e9817259bcafd7b267f9ba_saturation_sat3.0.jpg with annotation.
Uploaded 16_jpg.rf.368ef810cdcaa4e377bff7e40cd31dae_saturation_sat3.0.jpg with annotation.
Uploaded 02a05622ca_jpg.rf.578ae697af2fecbcf25608609599b31c_saturation_sat3.0.jpg with annotation.
Uploaded ee8bc44ec0_jpg.rf.52e7f5d7449ddbcaa8e0549277a1b52b_saturation_sat3.0.jpg with annotation.
Uploaded 549937246d_jpg.rf.04a4389c4bb31de485ed34e53900807f_saturation_sat3.0.jpg with annotation.
Uploaded 880_jpg.rf.bc2f5583ba557ebf6548f3f48c83f186_saturation_sat3.0.jpg with annotation.
Uploaded 763c9e5d99_jpg.rf.fc97b641d02f1baaecbd4dbfcaf20842_saturation_sat3.0.jpg with annotation.
Uploaded bf62dc17a5_jpg.rf.387954e76181e3a6f165b414a018dd02_saturation_sat3.0.jpg with annotation.
Uploaded 2025-01-30_10-05-00_3621_jpeg.rf.6c7a3b5e4e7d063af9f6acda2c1495ec_saturation_sat3.0.jpg with annotation.
Uploa

KeyboardInterrupt: 